In [1]:
import pandas as pd
import numpy as np
from datetime import datetime as dt
import time
from sklearn.preprocessing import MultiLabelBinarizer
import itertools

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
df = pd.read_csv("res/anime.csv")
#df = df.sample(frac=0.2, random_state=3000)

#Create name refrence df
name_ref = df[["MAL_ID","Name", "English name"]]
df.set_index("MAL_ID", inplace=True)

df

,Name,Score,Genres,English name,Japanese name,Type,Episodes,Aired,Premiered,Producers,Licensors,Studios,Source,Duration,Rating,Ranked,Popularity,Members,Favorites,Watching,Completed,On-Hold,Dropped,Plan to Watch,Score-10,Score-9,Score-8,Score-7,Score-6,Score-5,Score-4,Score-3,Score-2,Score-1
MAL_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,Cowboy Bebop,8.78,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",Cowboy Bebop,カウボーイビバップ,TV,26,"Apr 3, 1998 to Apr 24, 1999",Spring 1998,Bandai Visual,"Funimation, Bandai Entertainment",Sunrise,Original,24 min. per ep.,R - 17+ (violence & profanity),28.0,39,1251960,61971,105808,718161,71513,26678,329800,229170.0,182126.0,131625.0,62330.0,20688.0,8904.0,3184.0,1357.0,741.0,1580.0
5,Cowboy Bebop: Tengoku no Tobira,8.39,"Action, Drama, Mystery, Sci-Fi, Space",Cowboy Bebop:The Movie,カウボーイビバップ 天国の扉,Movie,1,"Sep 1, 2001",Unknown,"Sunrise, Bandai Visual",Sony Pictures Entertainment,Bones,Original,1 hr. 55 min.,R - 17+ (violence & profanity),159.0,518,273145,1174,4143,208333,1935,770,57964,30043.0,49201.0,49505.0,22632.0,5805.0,1877.0,577.0,221.0,109.0,379.0
6,Trigun,8.24,"Action, Sci-Fi, Adventure, Comedy, Drama, Shounen",Trigun,トライガン,TV,26,"Apr 1, 1998 to Sep 30, 1998",Spring 1998,Victor Entertainment,"Funimation, Geneon Entertainment USA",Madhouse,Manga,24 min. per ep.,PG-13 - Teens 13 or older,266.0,201,558913,12944,29113,343492,25465,13925,146918,50229.0,75651.0,86142.0,49432.0,15376.0,5838.0,1965.0,664.0,316.0,533.0
7,Witch Hunter Robin,7.27,"Action, Mystery, Police, Supernatural, Drama, ...",Witch Hunter Robin,Witch Hunter ROBIN (ウイッチハンターロビン),TV,26,"Jul 2, 2002 to Dec 24, 2002",Summer 2002,"TV Tokyo, Bandai Visual, Dentsu, Victor Entert...","Funimation, Bandai Entertainment",Sunrise,Original,25 min. per ep.,PG-13 - Teens 13 or older,2481.0,1467,94683,587,4300,46165,5121,5378,33719,2182.0,4806.0,10128.0,11618.0,5709.0,2920.0,1083.0,353.0,164.0,131.0
8,Bouken Ou Beet,6.98,"Adventure, Fantasy, Shounen, Supernatural",Beet the Vandel Buster,冒険王ビィト,TV,52,"Sep 30, 2004 to Sep 29, 2005",Fall 2004,"TV Tokyo, Dentsu",Unknown,Toei Animation,Manga,23 min. per ep.,PG - Children,3710.0,4369,13224,18,642,7314,766,1108,3394,312.0,529.0,1242.0,1713.0,1068.0,634.0,265.0,83.0,50.0,27.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48481,Daomu Biji Zhi Qinling Shen Shu,Unknown,"Adventure, Mystery, Supernatural",Unknown,盗墓笔记之秦岭神树,ONA,Unknown,"Apr 4, 2021 to ?",Unknown,Unknown,Unknown,Unknown,Novel,Unknown,Unknown,Unknown,13116,354,6,0,0,0,0,354,Unknown,Unknown,Unknown,1.0,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown
48483,Mieruko-chan,Unknown,"Comedy, Horror, Supernatural",Unknown,見える子ちゃん,TV,Unknown,2021 to ?,Unknown,Kadokawa,Unknown,Passione,Manga,Unknown,Unknown,Unknown,17562,7010,70,14,0,14,0,6982,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown
48488,Higurashi no Naku Koro ni Sotsu,Unknown,"Mystery, Dementia, Horror, Psychological, Supe...",Higurashi:When They Cry – SOTSU,ひぐらしのなく頃に卒,TV,Unknown,"Jul, 2021 to ?",Summer 2021,Unknown,Unknown,Unknown,Visual novel,Unknown,R - 17+ (violence & profanity),Unknown,17558,11309,48,3,0,0,0,11306,1.0,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown


## Data Wrangling

In [4]:
#drop target variables
#features = df.drop(["Score", *["Score-"+str(i) for i in range(1,11)], "Ranked", "Popularity"], axis=1)

#drop irrelivant variables
#irr_ind = features.columns.get_loc("Members")
#features.drop([*features.columns[irr_ind:], "Japanese name", "Premiered", "Name", "English name"], axis=1, inplace=True)

features = df
features.drop(["Japanese name", "Premiered", "English name"], axis=1, inplace=True)

In [5]:
#simplify Rating
def simplify_rating(rating):
    
    rating = str(rating)
    if "nan" in rating:
        return np.nan
    
    if rating.startswith("R"):
        if rating.startswith("Rx"):
            return "Rx"
        if rating.startswith("R+"):
            return "R+"
        return "R"
    
    if rating.startswith("PG"):
        if "13" in rating:
            return"PG13"
        return "PG"
    
    if rating.startswith("G"):
        return "G"

features["Rating"] = features["Rating"].apply(simplify_rating)

In [6]:
#One Hot Encode List Columns
list_cols = ["Genres","Licensors","Producers", "Studios"]
for column in list_cols:
    #Split into lists and clean strings
    features[column] = features[column].apply(lambda x: [_.strip() for _ in x.split(",")])
    gen_lists = features[column].copy().dropna().to_list()
    gen_unq = list(set(list(itertools.chain(*gen_lists))))
    
    mlb = MultiLabelBinarizer()
    mlb.fit([gen_unq])
    
    one_hot = pd.DataFrame(mlb.fit_transform(features[column]),columns=[f"{column}_{i}" for i in mlb.classes_], index=features.index)
    features = pd.concat([features,one_hot], axis=1, join='outer')
    
#One Hot Encode Categorical Columsn
cat_cols = ["Type","Source","Rating"]
features = pd.concat([features, pd.get_dummies(features[cat_cols])], axis=1, join='outer')

In [7]:
#Drop original columns
og_cols = list_cols+cat_cols
features.drop(og_cols, axis=1, inplace=True)

In [8]:
#drop unknown columns and data
for column in features:
    if "Unknown" in column:
        features.drop([column], axis=1, inplace=True)

features.replace(["Unknown", None], np.nan, inplace=True)

In [9]:
#convert strings to datetime objects
def str_to_dt(string):
    string = str(string)
    string_list = [string]
    if "to" in string:
        string_list = string.split("to")
        
    dts = []
    for i in string_list:
        i = i.strip()
        if i=="?" or i=="nan" or pd.isnull(i):
            string_dt = np.nan 
        else:
            try:
                string_dt = dt.strptime(i, "%b %d, %Y")
            except:
                try:
                    string_dt = dt.strptime(i, "%b, %Y")
                except:
                    string_dt = dt.strptime(i, "%Y")
        dts.append(string_dt)
    return dts

features["Aired"] = features["Aired"].apply(str_to_dt)
features[['Aired_Start','Aired_End']] = pd.DataFrame(features.Aired.tolist(), index= features.index)
features.drop(["Aired"], axis=1, inplace=True)

In [10]:
#convert datetime objects to unix time
features["Aired_Start_Unix"] = features["Aired_Start"].apply(lambda x: x.timestamp() if not pd.isnull(x) else np.nan)
features["Aired_End_Unix"] = features["Aired_End"].apply(lambda x: x.timestamp() if not pd.isnull(x) else np.nan)

#features[['Aired_Start','Aired_End',"Aired_Start_Unix", "Aired_End_Unix"]]

In [11]:
#convert duration into int
def str_to_num(string):
    
    string = str(string)
    if "nan" in string:
        return np.nan
    
    string_lst = string.split(".")
    minute = 0
    hour = 0
    
    for i in string_lst:
        if "min" in i:
            minute += int(i.strip("min "))
        if "hr" in i:
            hour += int(i.strip("hr "))
    
    return minute+(60*hour)

features["Duration"] = features["Duration"].apply(str_to_num)

In [12]:
#fill null values
    #fill episodes with rounded mean
features["Episodes"] = features["Episodes"].apply(lambda x: int(x) if not pd.isnull(x) else np.nan)
features["Episodes"] = features["Episodes"].fillna(int(features["Episodes"].mean()))

    #fill duration with mode
features["Duration"] = features["Duration"].apply(lambda x: int(x) if not pd.isnull(x) else np.nan)
features["Duration"] = features["Duration"].fillna(int(features["Duration"].mode()))

    #fill aired dates with each other if only one is null or drop row if both are null
features["Aired_End_Unix"] = features["Aired_End_Unix"].fillna(features["Aired_Start_Unix"])
features["Aired_Start_Unix"] = features["Aired_Start_Unix"].fillna(features["Aired_End_Unix"])
features.dropna(subset = ["Aired_Start_Unix", "Aired_End_Unix"], inplace=True)

In [13]:
features.drop(["Aired_Start","Aired_End"], axis=1).isnull().sum()

Name                   0
Score               4839
Episodes               0
Duration               0
Ranked              1584
                    ... 
Rating_R               0
Rating_R+              0
Rating_Rx              0
Aired_Start_Unix       0
Aired_End_Unix         0
Length: 2201, dtype: int64

In [14]:
features

Name Score  Episodes  Duration   Ranked  \
MAL_ID                                                                       
1                          Cowboy Bebop  8.78      26.0      24.0     28.0   
5       Cowboy Bebop: Tengoku no Tobira  8.39       1.0     115.0    159.0   
6                                Trigun  8.24      26.0      24.0    266.0   
7                    Witch Hunter Robin  7.27      26.0      25.0   2481.0   
8                        Bouken Ou Beet  6.98      52.0      23.0   3710.0   
...                                 ...   ...       ...       ...      ...   
48480       Wan Jie Shen Zhu 3rd Season   NaN      48.0       9.0  12855.0   
48481   Daomu Biji Zhi Qinling Shen Shu   NaN      11.0      24.0      NaN   
48483                      Mieruko-chan   NaN      11.0      24.0      NaN   
48488   Higurashi no Naku Koro ni Sotsu   NaN      11.0      24.0      NaN   
48492                     Scarlet Nexus   NaN      11.0      24.0      NaN   

        Popularity  Members  Favorites  Watching  Completed  On-Hold  Dropped  \
MAL_ID                                                                          
1               39  1251960      61971    105808     718161    71513    26678   
5              518   273145       1174      4143     208333     1935      770   
6              201   558913      12944     29113     343492    25465    13925   
7             1467    94683        587      4300      46165     5121     5378   
8             4369    13224         18       642       7314      766     1108   
...            ...      ...        ...       ...        ...      ...      ...   
48480        16457      125          0        36          0        5        5   
48481        13116      354          6         0          0        0        0   
48483        17562     7010         70        14          0       14        0   
48488        17558    11309         48         3          0        0        0   
48492        17563      310          0         4          0        1        1   

        Plan to Watch  Score-10   Score-9   Score-8  Score-7  Score-6 Score-5  \
MAL_ID                                                                          
1              329800  229170.0  182126.0  131625.0  62330.0  20688.0  8904.0   
5               57964   30043.0   49201.0   49505.0  22632.0   5805.0  1877.0   
6              146918   50229.0   75651.0   86142.0  49432.0  15376.0  5838.0   
7               33719    2182.0    4806.0   10128.0  11618.0   5709.0  2920.0   
8                3394     312.0     529.0    1242.0   1713.0   1068.0   634.0   
...               ...       ...       ...       ...      ...      ...     ...   
48480              79       NaN       NaN       2.0      1.0      1.0     NaN   
48481             354       NaN       NaN       NaN      1.0      NaN     NaN   
48483            6982       NaN       NaN       NaN      NaN      NaN     NaN   
48488           11306       1.0       NaN       NaN      NaN      NaN     NaN   
48492             304       NaN       NaN       NaN      NaN      NaN     NaN   

       Score-4 Score-3 Score-2 Score-1  Genres_Action  Genres_Adventure  \
MAL_ID                                                                    
1       3184.0  1357.0   741.0  1580.0              1                 1   
5        577.0   221.0   109.0   379.0              1                 0   
6       1965.0   664.0   316.0   533.0              1                 1   
7       1083.0   353.0   164.0   131.0              1                 0   
8        265.0    83.0    50.0    27.0              0                 1   
...        ...     ...     ...     ...            ...               ...   
48480      NaN     NaN     NaN     NaN              0                 1   
48481      NaN     NaN     NaN     NaN              0                 1   
48483      NaN     NaN     NaN     NaN              0                 0   
48488      NaN     NaN     NaN     NaN              0                 0   
48492      N

In [15]:
features.to_csv("res/anime_wrangled.csv")